In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [131]:
import torch
from comm_agents.data.datahandler import RefExpDataset
from comm_agents.models.model_single_enc import SingleEncModel
import pandas as pd
import plotly.graph_objects as go


In [132]:
DATA_PATH = './data/reference_experiment_dat_1000_new_new_new2.csv'

In [133]:
df = pd.read_csv(DATA_PATH)
df.loc[:, ['alpha_star0', 'alpha_star1', 'phi_star0', 'phi_star1']] = \
    df[['alpha_star0', 'alpha_star1', 'phi_star0', 'phi_star1']] / np.pi
df['q0_t_q1'] = df.q0 * df.q1
df['imp_0'] = df.v_ref_b * df.m0 / (df.m_ref_b + df.m0)
df['imp_1'] = df.v_ref_b * df.m1 / (df.m_ref_b + df.m1)
df.head(9)

,m0,m1,q0,q1,o_a_0_0,o_a_0_1,o_a_0_2,o_a_0_3,o_a_0_4,o_a_0_5,...,v_ref_a,m_ref_b,v_ref_b,alpha_star0,alpha_star1,phi_star0,phi_star1,q0_t_q1,imp_0,imp_1
0,2.793947e-20,3.673120e-20,1.144524e-16,-1.172200e-15,0.0,0.013870,0.027740,0.041611,0.055481,0.069351,...,1.662326,2.000000e-20,1.575615,0.085692,0.127713,0.533260,0.535582,-1.341611e-31,0.918280,1.020148
1,3.446018e-20,2.224602e-20,1.083617e-16,-1.284728e-15,0.0,0.013790,0.027580,0.041370,0.055160,0.068950,...,1.877513,2.000000e-20,1.326676,0.085692,0.050304,0.551302,0.547630,-1.392153e-31,0.839467,0.698604
2,2.245701e-20,2.010961e-20,1.397350e-16,-1.759350e-15,0.0,0.013069,0.026139,0.039208,0.052278,0.065347,...,1.387216,2.000000e-20,1.937362,0.096823,0.085692,0.539148,0.539160,-2.458428e-31,1.024739,0.971328
3,3.445829e-20,2.901352e-20,1.237884e-16,-1.625558e-15,0.0,0.012202,0.024403,0.036605,0.048807,0.061008,...,1.661205,2.000000e-20,1.287810,0.115550,0.090085,0.580853,0.577058,-2.012251e-31,0.814857,0.762318
4,2.568873e-20,4.085563e-20,1.015901e-16,-1.022468e-15,0.0,0.014597,0.029194,0.043792,0.058389,0.072986,...,1.667318,2.000000e-20,1.061019,0.077158,0.152729,0.556564,0.563520,-1.038726e-31,0.596564,0.712319
5,2.136741e-20,1.634227e-20,1.443138e-16,-1.737278e-15,0.0,0.016404,0.032807,0.049211,0.065614,0.082018,...,1.696430,2.000000e-20,1.950191,0.059320,0.045221,0.539152,0.540184,-2.507132e-31,1.007327,0.876955
6,2.896279e-20,3.471084e-20,1.053652e-16,-1.004226e-15,0.0,0.011877,0.023753,0.035630,0.047506,0.059383,...,1.453773,2.000000e-20,1.119145,0.120379,0.167956,0.552524,0.555276,-1.058105e-31,0.662004,0.710032
7,3.330262e-20,2.743843e-20,1.285886e-16,-1.418881e-15,0.0,0.011587,0.023174,0.034761,0.046348,0.057935,...,1.544036,2.000000e-20,1.722669,0.130179,0.099106,0.539373,0.537860,-1.824519e-31,1.076296,0.996393
8,2.510125e-20,1.794731e-20,1.155914e-16,-1.100386e-15,0.0,0.014764,0.029528,0.044292,0.059056,0.073820,...,1.664679,2.000000e-20,1.712311,0.075080,0.052053,0.526330,0.525952,-1.271952e-31,0.952992,0.809843


In [64]:
import plotly.express as px
import numpy as np
fig = px.scatter_matrix(df[['m0', 'v_ref_a']], color=df.alpha_star0, opacity=1)
fig.show()

In [125]:
fig = px.scatter_3d(df, x=df.m0, y=df.v_ref_a, z=df.alpha_star0, color=df.alpha_star0)
fig.show()

In [126]:
fig = px.scatter_matrix(df[['m0', 'v_ref_b', 'imp_0', 'q0_t_q1']],
                        color=df.phi_star1, opacity=.5)
fig.show()

In [130]:
fig = px.scatter_3d(df, x=df.v_ref_b, y=df.phi_star0, z=df.q0_t_q1, color=df.phi_star0)
fig.show()

In [135]:
t = np.linspace(0, 10, 10)
fig = go.Figure()
for i in range(100):
    y = df[[c for c in df.columns if 'o_a_1' in c]][i:i+1].values[0]
    trace = go.Scatter(x=t, y=y, mode='lines+markers', opacity=.1)
    fig.add_trace(trace)
fig.show()

In [152]:
np.random.choice(range(len(df)), 100)

array([1828,  462, 2085, 1226, 1682, 1294, 1240,  331, 1301, 1818,   83,
       1551, 2154, 1910, 2024, 1030, 2434,  235,  533, 2474, 2011,  274,
        688, 1209, 2117, 2589,   74,  175, 1171,  221, 2154, 1582,  902,
       1801, 2010, 1827,  220,  709, 2199, 2326, 1158, 1249,  815, 2158,
       2330, 1705,  462, 2488,  252, 1667, 2169, 1032, 2557, 1847, 2645,
       1110,  630,  883,  785, 2243,  912,  971, 1894, 1339,  253, 1793,
       2395,  928,  711, 2407, 2429,  130,  661,  972,  614, 2088, 2581,
       2275, 2051, 2393, 1529, 1243, 1795,  835,  292, 2123,  269, 1149,
         47,  205, 1996,  504, 2220, 1954, 1494, 2179,  631, 1305, 1139,
       1288])

In [157]:
t = np.linspace(0, 10, 10)
fig = go.Figure()
num_samples = 100
for i in np.random.choice(range(len(df)), num_samples):
    y = df[[c for c in df.columns if 'o_b_0' in c]][i:i+1].values[0]
    trace = go.Scatter(x=t, y=y, mode='lines+markers', opacity=.1)
    fig.add_trace(trace)
fig.show()

In [158]:
df.columns

Index(['m0', 'm1', 'q0', 'q1', 'o_a_0_0', 'o_a_0_1', 'o_a_0_2', 'o_a_0_3',
       'o_a_0_4', 'o_a_0_5', 'o_a_0_6', 'o_a_0_7', 'o_a_0_8', 'o_a_0_9',
       'o_a_1_0', 'o_a_1_1', 'o_a_1_2', 'o_a_1_3', 'o_a_1_4', 'o_a_1_5',
       'o_a_1_6', 'o_a_1_7', 'o_a_1_8', 'o_a_1_9', 'o_b_0_0', 'o_b_0_1',
       'o_b_0_2', 'o_b_0_3', 'o_b_0_4', 'o_b_0_5', 'o_b_0_6', 'o_b_0_7',
       'o_b_0_8', 'o_b_0_9', 'o_b_1_0', 'o_b_1_1', 'o_b_1_2', 'o_b_1_3',
       'o_b_1_4', 'o_b_1_5', 'o_b_1_6', 'o_b_1_7', 'o_b_1_8', 'o_b_1_9',
       'm_ref_a', 'v_ref_a', 'm_ref_b', 'v_ref_b', 'alpha_star0',
       'alpha_star1', 'phi_star0', 'phi_star1', 'q0_t_q1', 'imp_0', 'imp_1'],
      dtype='object')

In [82]:
df[[c for c in df.columns if 'o0_a' in c]][0:0+1].values

array([[0.        , 0.0138702 , 0.02774041, 0.04161061, 0.05548082,
        0.06935102, 0.08322123, 0.09709143, 0.11096164, 0.12483184]])